In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def plot_grouped_bar_chart(data, bar_labels, group_labels, title="", xlabel="", ylabel="", bar_width=0.35, figsize=None, legend=True, font_size=None, use_bar_labels=False):
    if font_size is not None:
        plt.rcParams.update({'font.size': font_size})
        
    fig = plt.figure(figsize=figsize)
    
    num_bars_per_group = len(data)
    num_groups = len(data[0])

    y = np.arange(num_groups)

    cmap = plt.get_cmap('jet')
    colors = cmap(np.linspace(0, 1.0, num_bars_per_group))
    hatches = ['/', '\\' , '|', '-', '+', 'x', 'o', 'O', '.', '*',
               '//', '\\\\' , '||', '--', '++', 'xx', 'oo', 'O0', '..', '**']
    for bar_n in range(num_bars_per_group):
        bars = plt.barh(
            y + bar_n * bar_width, 
            width=data[bar_n], 
            height=bar_width, 
            label=bar_labels[bar_n], 
            hatch=hatches[bar_n], 
            color=colors[bar_n], 
            edgecolor='black'
        )
        
        if use_bar_labels:
            plt.bar_label(bars, labels=np.round(data[bar_n], 4), padding=8)
        

    plt.xlim(left=0.9)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    #plt.title(title)
    plt.yticks(y + bar_width * (num_bars_per_group - 1) / 2, group_labels, rotation=90)
    if legend:
        plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
tables = pd.read_csv("tables.csv")

In [ ]:
tables

In [ ]:
def normalize_within_group(df):
    def normalize(group):
        min_val = group.min()
        max_val = group.max()
        return 1 + (group - min_val) * (2 - 1) / (max_val - min_val)

    df['normalized_result'] = df.groupby(['machine', 'bench'])['result'].transform(normalize)

    return df

In [ ]:
normalize_within_group(tables)
tables

In [ ]:
machines = list(tables['machine'].unique())
benches = list(tables['bench'].unique())
locks = list(tables['lock'].unique())

print(f'Machines:  {machines}')
print(f'Benches: {benches}')
print(f'Locks: {locks}')

In [ ]:
grouped_data = dict()

for bench in benches:
    grouped_data[bench] = dict()
    for lock in locks:
        grouped_data[bench][lock] = list()
        for machine in machines:
            datum = tables.loc[(tables.bench == bench) & (tables.machine == machine) & (tables.lock == lock)].normalized_result
            if datum.size > 0:
                grouped_data[bench][lock].append(datum.values[0])
            else:
                grouped_data[bench][lock].append(0)
        
print(grouped_data)

In [ ]:
plot_grouped_bar_chart(
    [grouped_data['locks'][lock] for lock in locks],
    locks,
    machines,
    "Normalized Benchmark Performance (Locks)",
    "Elapsed time, normalized",
    "Machines",
    bar_width=0.06,
    figsize=(10, 30),
    legend=False,
    font_size=35
)

In [ ]:
plot_grouped_bar_chart(
    [grouped_data['sets'][lock] for lock in locks],
    locks,
    machines,
    "Normalized Benchmark Performance (Sets)",
    "Elapsed time, normalized",
    "Machines",
    bar_width=0.06,
    figsize=(10, 30),
    legend=False,
    font_size=35
)

In [ ]:
plot_grouped_bar_chart(
    [grouped_data['maps'][lock] for lock in locks],
    locks,
    machines,
    "Normalized Benchmark Performance (Maps)",
    "Elapsed time, normalized",
    "Machines",
    bar_width=0.06,
    figsize=(10, 30),
    legend=False,
    font_size=35
)